In [1]:

from importlib import reload
import os
import pandas as pd
from io import StringIO
import itertools
import ipdb
import datetime
from collections import Counter

import h5py
import json
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib
print(tf.__version__)

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import mytf.s3utils as msu
import mytf.utils as mu
import mytf.validation as mv
import mytf.plot as mp

1.14.0


Using TensorFlow backend.


In [2]:
!pip search tensorflow # oh wow so 2.0 is available to upgradE? hmm

tensorflow (2.0.0)                                - TensorFlow is an open
                                                    source machine learning
                                                    framework for everyone.
  INSTALLED: 1.14.0
  LATEST:    2.0.0
tensorflow-qndex (0.0.22)                         - tensorflow-qnd x
                                                    tensorflow-extenteten
tensorflow-estimator (2.1.0)                      - TensorFlow Estimator.
  INSTALLED: 1.14.0
  LATEST:    2.1.0
mesh-tensorflow (0.1.7)                           - Mesh TensorFlow
tensorflow-io (0.10.0)                            - TensorFlow IO
tensorflow-ops (0.0.0)                            - tensorflow-ops
tensorflow-addons (0.6.0)                         - TensorFlow Addons.
tensorflow-plot (0.3.2)                           - TensorFlow Plot
tensorflow-lattice (0.9.9)                        - TensorFlow Lattice
                                                    provides lattice

In [3]:
tf.enable_eager_execution()

In [4]:
# Use datasets from 
# 2019-12-25.ipynb
# 
# train ... new datasets, ...
outdir = 'history/2019-12-22T174803Z'
train_loc = f'{outdir}/train_balanced.h5'
test_loc = f'{outdir}/test_balanced.h5'
train_shuff_loc = f'{outdir}/train_scaled_balanced_shuffled.h5'
print(mu.h5_keys(train_shuff_loc))
print(mu.h5_keys(test_loc))

X, Ylabels = mu.read_h5_two(
                source_location=train_shuff_loc, 
                Xdataset=f'X',
                Ydataset=f'Ylabels')

['X', 'Ylabels']
['X_0', 'X_1', 'X_2', 'X_3', 'Ylabels_0', 'Ylabels_1', 'Ylabels_2', 'Ylabels_3']


In [5]:
# Working dir... for new model
save_dir = 'history'
ts = mu.quickts(); print('starting,', ts)

size = X.shape[0]
workdir = f'{save_dir}/{ts}'
os.mkdir(workdir)
print(f'Made new workdir, {workdir}')

starting, 2019-12-29T000509Z
Made new workdir, history/2019-12-29T000509Z


In [8]:
# Parameters..

# 'batch_size'
BATCH_SIZE = 32
EPOCHS = 10
# 
# ValueError: If a RNN is stateful, it needs to know its batch size. Specify the batch size of your input tensors: 
# - If using a Sequential model, specify the batch size by passing a `batch_input_shape` argument to your first layer.
# - If using the functional API, specify the batch size by passing a `batch_shape` argument to your Input layer.

lstm_params = [{
    'units': 64,
    'dropout': 0.6,
    'recurrent_dropout': 0.6,
    'batch_input_shape': (None, 64, 8),
    },

]

optimizer_params = {
    'learning_rate': 0.001,  
    'beta1': 0.9, 
    'beta2': 0.999, 
    'epsilon': 1e-08
}

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(**lstm_params[0]),
    # 4 because 'A', 'B', 'C', 'D'.
    tf.keras.layers.Dense(4)])

# save base unfitted model.
mu.save_model(model=model, 
              loc=f'{workdir}/00000__unfitted_model.h5')

W1229 00:14:51.874383 139851324434240 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1229 00:14:51.900942 139851324434240 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1229 00:14:52.016719 139851324434240 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1229 00:14:52.035298 139851324434240 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1229 00:14:52.054618 139851324434240 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [18]:
print(workdir)

history/2019-12-29T000509Z


In [20]:
reload(mv)

<module 'mytf.validation' from '/home/ec2-user/SageMaker/aviation-pilot-physiology-hmm/mytf/validation.py'>

In [21]:
size

34980

In [22]:
%%time

model = mu.load_model(f'{workdir}/00000__unfitted_model.h5')

class_weights = {0: 1., 1: 1., 2: 1., 3: 1.}
dataset_batches = mu.build_dataset_weighty_v3(
        {'x_train': X,
         'ylabels_train': Ylabels.astype('int64')},
        list(range(size)), 
        class_weights,
        batch_size=BATCH_SIZE)
    
#with ipdb.launch_ipdb_on_exception():
mu.do_train(
        model,
        dataset_batches,
        k=size,
        epochs=EPOCHS,
        optimizer_params=optimizer_params,
        saveloc=workdir)

W1229 00:53:09.156462 139851324434240 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.
W1229 00:53:09.503507 139851324434240 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/aviation-pilot-physiology-hmm/mytf/utils.py:241: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

0it [00:00, ?it/s]W1229 00:53:09.909878 139851324434240 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1094it [07:37,  2.39it/s]
1094it [07:41,  2.37it/s]
1094it [07:41,  2.37it/s]
1094it [07:43,  2.36it/s]
1094it [07:43,  2.36it/s]
1094it [07:38,  2.38it/s]
1094it [07:39,

CPU times: user 1h 16min 56s, sys: 5.01 s, total: 1h 17min 1s
Wall time: 1h 16min 53s


In [25]:
test_loc

'history/2019-12-22T174803Z/test_balanced.h5'

In [ ]:
print('starting validation', mu.quickts())
batch_losses_vec = []

for epoch in range(10):
    for batch in tqdm([0, 100, 200, 300, 400, 500, 600, 
                      700, 800, 900, 1000, 1090]):
        step = batch
        prefix = (f'{workdir}/epoch_{str(epoch).zfill(3)}'
                               f'_batch_{str(batch).zfill(5)}')

        modelname = f'{prefix}_model.h5'
        #print(modelname, os.path.exists(modelname))
        
        steploss = mv.perf_wrapper(modelname,
                                   dataloc=test_loc,
                                   eager=True,
                                  batch_size=32)
        batch_losses_vec.append([float(x) for x in steploss])
        mv.json_save({'batch_losses_vec': batch_losses_vec,
                      'step': int(step)
                  }, 
                  f'{prefix}_validation_losses.json')
    
print('done validation', mu.quickts())
#####
lossesarr = np.array(batch_losses_vec)
meanlossesarr = np.mean(lossesarr, axis=1)

batch_losses_vec[:5]
#batch_losses_vec = []
#for step in np.arange(0, 1068, 10):
# [2.8359528, 0.45356295, 1.7049086, 4.099845]

plt.plot([x[0] for x in batch_losses_vec], color='blue', label='0')
plt.plot([x[1] for x in batch_losses_vec], color='green', label='1')
plt.plot([x[2] for x in batch_losses_vec], color='red', label='2')
plt.plot([x[3] for x in batch_losses_vec], color='orange', label='3')
plt.plot(meanlossesarr, color='black', label='mean')
plt.title(f'validation losses  (model {ts})')
plt.legend()     
        

  0%|          | 0/12 [00:00<?, ?it/s]

starting validation 2019-12-29T062045Z


W1229 06:20:46.112020 139851324434240 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.
W1229 06:20:46.749158 139851324434240 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/aviation-pilot-physiology-hmm/mytf/validation.py:23: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

 67%|██████▋   | 8/12 [52:04<25:59, 389.88s/it]W1229 15:02:23.900037 139851324434240 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.
